In [249]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,confusion_matrix
import matplotlib.pyplot as plt
from sklearn.externals.six import StringIO
import pydotplus
from IPython.display import Image
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from collections import Counter
from xgboost import XGBClassifier
from xgboost import plot_tree
from sklearn import svm
import graphviz
import matplotlib as matplot
import seaborn as sns
%matplotlib inline

In [250]:
df=pd.read_csv("C:/Users/steve/Desktop/hr data/b.csv")
print(df)

      Attrition  Age  BusinessTravel  DailyRate  Department  DistanceFromHome  \
0            60   41              50       1102          30                 1   
1            40   49             100        279          60                 8   
2            60   37              50       1373          60                 2   
3            40   33             100       1392          60                 3   
4            40   27              50        591          60                 2   
5            40   32             100       1005          60                 2   
6            40   59              50       1324          60                 3   
7            40   30              50       1358          60                24   
8            40   38             100        216          60                23   
9            40   36              50       1299          60                27   
10           40   35              50        809          60                16   
11           40   29        

In [251]:
df.isnull().any()

Attrition                   False
Age                         False
BusinessTravel              False
DailyRate                   False
Department                  False
DistanceFromHome            False
Education                   False
EnvironmentSatisfaction     False
Gender                      False
HourlyRate                  False
JobInvolvement              False
JobLevel                    False
JobSatisfaction             False
MaritalStatus               False
MonthlyIncome               False
MonthlyRate                 False
NumCompaniesWorked          False
OverTime                    False
PercentSalaryHike           False
PerformanceRating           False
RelationshipSatisfaction    False
StockOptionLevel            False
TotalWorkingYears           False
TrainingTimesLastYear       False
WorkLifeBalance             False
YearsAtCompany              False
YearsInCurrentRole          False
YearsSinceLastPromotion     False
YearsWithCurrManager        False
dtype: bool

In [252]:
df.shape

(1470, 29)

In [253]:
attrition_rate = df.Attrition.value_counts() / len(df)
print(attrition_rate)

40    0.838776
60    0.161224
Name: Attrition, dtype: float64


In [254]:
df.describe()


,Attrition,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EnvironmentSatisfaction,Gender,HourlyRate,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,...,1470.00000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000
mean,43.224490,36.923810,54.319728,802.485714,52.183673,9.192517,58.258503,54.435374,52.000000,65.891156,...,63.07483,2.712245,0.793878,11.279592,2.799320,55.224490,7.008163,4.229252,2.187755,4.123129
std,7.357261,9.135373,26.608496,403.509100,15.833769,8.106864,20.483299,21.861644,9.801293,20.329428,...,7.21647,1.081209,0.852077,7.780782,1.289271,14.129517,6.126525,3.623137,3.222430,3.568136
min,40.000000,18.000000,0.000000,102.000000,30.000000,1.000000,20.000000,20.000000,40.000000,30.000000,...,60.00000,1.000000,0.000000,0.000000,0.000000,20.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,30.000000,50.000000,465.000000,30.000000,2.000000,40.000000,40.000000,40.000000,48.000000,...,60.00000,2.000000,0.000000,6.000000,2.000000,40.000000,3.000000,2.000000,0.000000,2.000000
50%,40.000000,36.000000,50.000000,802.000000,60.000000,7.000000,60.000000,60.000000,60.000000,66.000000,...,60.00000,3.000000,1.000000,10.000000,3.000000,60.000000,5.000000,3.000000,1.000000,3.000000
75%,40.000000,43.000000,50.000000,1157.000000,60.000000,14.000000,80.000000,80.000000,60.000000,83.750000,...,60.00000,4.000000,1.000000,15.000000,3.000000,60.000000,9.000000,7.000000,3.000000,7.000000
max,60.000000,60.000000,100.000000,1499.000000,90.000000,29.000000,100.000000,80.000000,60.000000,100.000000,...,80.00000,4.000000,3.000000,40.000000,6.000000,80.000000,40.000000,18.000000,15.000000,17.000000


In [255]:
attrition_Summary = df.groupby('Attrition')
attrition_Summary.mean()

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
Attrition,,,,,,,,,,,,,,,,,,,,,
40,37.561233,52.838605,812.504461,52.627737,8.915653,58.540146,55.425791,51.873479,65.952149,55.409570,...,63.065693,2.733982,0.845093,11.862936,2.832928,55.620438,7.369019,4.484185,2.234388,4.367397
60,33.607595,62.025316,750.362869,49.873418,10.632911,56.793249,49.282700,52.658228,65.573840,50.379747,...,63.122363,2.599156,0.527426,8.244726,2.624473,53.164557,5.130802,2.902954,1.945148,2.852321


In [256]:
df.corr()

,Attrition,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EnvironmentSatisfaction,Gender,HourlyRate,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
Attrition,1.000000,-0.159205,0.127006,-0.056652,-0.063991,0.077924,-0.031373,-0.103369,0.029453,-0.006846,...,0.002889,-0.045872,-0.137145,-0.171063,-0.059478,-0.063939,-0.134392,-0.160545,-0.033019,-0.156199
Age,-0.159205,1.000000,-0.011807,0.010661,0.031882,-0.001686,0.208034,0.010146,-0.036311,0.024287,...,0.001904,0.053535,0.037510,0.680381,-0.019621,-0.021490,0.311309,0.212901,0.216513,0.202089
BusinessTravel,0.127006,-0.011807,1.000000,-0.015539,0.002640,-0.009696,-0.008670,-0.011310,-0.044896,-0.004164,...,0.001683,0.008926,-0.028257,0.007972,0.016357,0.004209,0.005212,-0.005336,0.005222,-0.000229
DailyRate,-0.056652,0.010661,-0.015539,1.000000,-0.007109,-0.004985,-0.016806,0.018355,-0.011716,0.023381,...,0.000473,0.007846,0.042143,0.014515,0.002453,-0.037848,-0.034055,0.009932,-0.033229,-0.026363
Department,-0.063991,0.031882,0.002640,-0.007109,1.000000,-0.017225,-0.007996,0.019395,0.041583,0.004144,...,0.024604,0.022414,0.012193,0.015762,-0.036875,-0.026383,-0.022920,-0.056315,-0.040061,-0.034282
DistanceFromHome,0.077924,-0.001686,-0.009696,-0.004985,-0.017225,1.000000,0.021042,-0.016075,-0.001851,0.031131,...,0.027110,0.006557,0.044872,0.004628,-0.036942,-0.026556,0.009508,0.018845,0.010029,0.014406
Education,-0.031373,0.208034,-0.008670,-0.016806,-0.007996,0.021042,1.000000,-0.027128,-0.016547,0.016775,...,-0.024539,-0.009118,0.018422,0.148280,-0.025100,0.009819,0.069114,0.060236,0.054254,0.069065
EnvironmentSatisfaction,-0.103369,0.010146,-0.011310,0.018355,0.019395,-0.016075,-0.027128,1.000000,0.000508,-0.049857,...,-0.029548,0.007665,0.003432,-0.002693,-0.019359,0.027627,0.001458,0.018007,0.016194,-0.004999
Gender,0.029453,-0.036311,-0.044896,-0.011716,0.041583,-0.001851,-0.016547,0.000508,1.000000,-0.000478,...,-0.013859,0.022868,0.012716,-0.046881,-0.038787,-0.002753,-0.029747,-0.041483,-0.026985,-0.030599
HourlyRate,-0.006846,0.024287,-0.004164,0.023381,0.004144,0.031131,0.016775,-0.049857,-0.000478,1.000000,...,-0.002172,0.001330,0.050263,-0.002334,-0.008548,-0.004607,-0.019582,-0.024106,-0.026716,-0.020123


In [257]:
emp_population = df['OverTime'][df['Attrition'] == 40].mean()
emp_turnover_Overtime = df[df['Attrition']==60]['OverTime'].mean()
print( 'The mean overtime for the employee population with no attrition is: ' + str(emp_population))
print( 'The mean overtime for employees that had a attrition is: ' + str(emp_turnover_Overtime) )

The mean overtime for the employee population with no attrition is: 2.34387672344
The mean overtime for employees that had a attrition is: 5.35864978903


In [258]:
print("Attrition count -        40 = No  60 = Yes")
print( df.Attrition.value_counts())
print("No. of people doing ovetime -        0 = No  10 = Yes")
print( df.OverTime.value_counts())


 Attrition count -        40 = No  60 = Yes
40    1233
60     237
Name: Attrition, dtype: int64
 No. of people doing ovetime -        0 = No  10 = Yes
0     1054
10     416
Name: OverTime, dtype: int64
